# Mangroves 
## 3.0.0 vs 4.0.0

Check list: 

    [x] compare the two versions in Maps/Terria
    [x] compare the two versions in Explorer
    [x] plot random tiles, check impact of cloud and shadow buffering on no data values. Years 1990, 2010, 2015, 2022.
     

In [ ]:
import datacube
import pandas as pd
import sys
import rioxarray as rioxr
import rasterio
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np

dc = datacube.Datacube(app="")

In [ ]:
# functions to use

def load_mangroves_3(tile, year):
    """
    load mangroves v3 over tile extent.
    """
    mangroves_3 = dc.load(
                product="ga_ls_mangrove_cover_cyear_3",
                region_code = tile, 
                time=year,
    )
    return mangroves_3

def load_mangroves_4(tile, year):
    """
    load mangroves v4 tiles from s3.
    """
    base_url = "s3://dea-public-data/derivative/ga_ls_mangrove_cover_cyear_3/4-0-0/{}/{}/{}--P1Y/ga_ls_mangrove_cover_cyear_3_{}_{}--P1Y_final_canopy_cover_class.tif"
    x_val = tile[:3]
    y_val = tile[-3:]
    url = base_url.format(x_val, y_val, year, tile, year)
    mangroves_3 = rioxr.open_rasterio(url)
    return mangroves_3

tile_lookup = pd.read_csv('grid_lookup.csv', sep=',', na_values=['NA'])
def lookup_old_value(new_value):
    """
    returns old region code because version 3 mangroves uses the old grid
    """
    result = tile_lookup[tile_lookup['new'] == new_value]
    result = result['old'].values[0]
    return result

def count_pixels(data):
    """
    returns count of unique pixels
    """
    unique_values, counts = np.unique(data, return_counts=True)
    unique_counts = dict(zip(unique_values, counts))
    return unique_counts

def print_pixel_info(tile, year):
    """
    Print pixel counts and percentage change between versions.
    """
    tile_old = lookup_old_value(tile)
    v3 = load_mangroves_3(tile_old, year)
    v4 = load_mangroves_4(tile, year)
    print('---------------------------------------------------------------------------------')
    print(f'{tile}')
    print('0 = not observed, 1 = woodland, 2 = open forest, 3 = closed forest, 225 = no data')
    print('---------------------------------------------------------------------------------')
    print('Pixel count version 3')
    print('---------------------------------------------------------------------------------')
    count_pixels_3 = count_pixels(v3.canopy_cover_class)
    for value, count in count_pixels_3.items():
        print(f'Value: {value:,}, Count: {count:,}')
    print('---------------------------------------------------------------------------------')
    print('Pixel count version 4')
    print('---------------------------------------------------------------------------------')
    count_pixels_4 = count_pixels(v4.values)
    for value, count in count_pixels_4.items():
        print(f'Value: {value:,}, Count: {count:,}')
    print('---------------------------------------------------------------------------------')
    print('Percentage change from version 3 to version 4')
    print('---------------------------------------------------------------------------------')
    for value in set(count_pixels_3.keys()).union(count_pixels_4.keys()):
        count_v3 = count_pixels_3.get(value, 0)
        count_v4 = count_pixels_4.get(value, 0)
        if count_v3 != 0:  # To avoid division by zero
            percent_change = ((count_v4 - count_v3) / count_v3) * 100
            print(f'Value: {value}, Percentage Change: {percent_change:.2f}%')
        else:
            print(f'Value: {value}, Percentage Change: Undefined (divided by zero)')
    sys.stdout.flush()
    
    
def plot_tile(tile, year):
    """
    Plot version 3 and 4 mangroves products next to each other.
    """
    tile_old = lookup_old_value(tile)
    v3 = load_mangroves_3(tile_old, year)
    v4 = load_mangroves_4(tile, year)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    img1 = ax1.imshow(v3.canopy_cover_class.squeeze(), cmap='Greens_r')
    img2 = ax2.imshow(v4.squeeze(), cmap='Greens_r')
    ax1.set_title(f'{tile_old} mangroves v3')
    ax2.set_title(f'{tile} mangroves v4')
    fig.colorbar(img1, ax=ax1)
    fig.colorbar(img2, ax=ax2)
    
    plt.show()
    plt.close()

def plot_tile_and_info(tile, year):
    """
    Print pixel info and plot mangroves products for a given tile and year.
    """
    print_pixel_info(tile, year)
    plot_tile(tile, year)
  

In [ ]:
tile_list = ['x27y45', 'x37y53', 'x44y58', 'x50y58', 'x57y58', 'x67y38', 'x65y33', 'x59y27', 'x51y34']

for tile in tile_list:
    plot_tile_and_info(tile, '1990')

In [ ]:
tile_list = ['x27y45', 'x37y53', 'x44y58', 'x50y58', 'x57y58', 'x67y38', 'x65y33', 'x59y27', 'x51y34']

for tile in tile_list:
    plot_tile_and_info(tile, '2010')

In [ ]:
tile_list = ['x27y45', 'x37y53', 'x44y58', 'x50y58', 'x57y58', 'x67y38', 'x65y33', 'x59y27', 'x51y34']

for tile in tile_list:
    plot_tile_and_info(tile, '2015')

In [ ]:
tile_list = ['x27y45', 'x37y53', 'x44y58', 'x50y58', 'x57y58', 'x67y38', 'x65y33', 'x59y27', 'x51y34']

for tile in tile_list:
    plot_tile_and_info(tile, '2022')